# LINEAR DECISION PROBLEM

The below example illustrates how to allocate money to different bond to maximize the total return. A company needs to invest 100.000$ in following collection of bonds to maximize the annual return. In order to solve the linear decision problem, we are going to use the library PuLP, which is an LP modeler written in python. PuLP can generate MPS or LP files and call GLPK, COIN CLP/CBC, CPLEX, and GUROBI to solve linear problems.

In [1]:
# Import require libraries

import pandas as pd
from pulp import *

In [2]:
# Create the dataframe from a dict

dict_investment = {'Bonds':['A', 'B', 'C', 'D', 'E'], 'Amount_Invested':['Xa', 'Xb','Xc', 'Xd', 'Xe'], 'Return': [0.095,0.08,0.09,0.09,0.09], 'Matuarity':['Long', 'Short', 'Long', 'Long', 'Short'], 'Risk':['High', 'Low', 'Low', 'High', 'High'], 'Tax-Free':['Yes', 'Yes', 'No', 'Yes', 'No']}

df_investment = pd.DataFrame(data=dict_investment)

In [3]:
df_investment

,Bonds,Amount_Invested,Return,Matuarity,Risk,Tax-Free
0,A,Xa,0.095,Long,High,Yes
1,B,Xb,0.080,Short,Low,Yes
2,C,Xc,0.090,Long,Low,No
3,D,Xd,0.090,Long,High,Yes
4,E,Xe,0.090,Short,High,No


In [4]:
# Convert the columns into numbers

df_investment['Risk'] = [1,0,0,1,1] # High:1 | Low:0
df_investment['Tax-Free'] = [1,1,0,1,0] # Yes:1 | No:0
df_investment['Matuarity'] = [0,1,0,0,1] # Short:1 | Long:0

df_investment

,Bonds,Amount_Invested,Return,Matuarity,Risk,Tax-Free
0,A,Xa,0.095,0,1,1
1,B,Xb,0.080,1,0,1
2,C,Xc,0.090,0,0,0
3,D,Xd,0.090,0,1,1
4,E,Xe,0.090,1,1,0


In [5]:
# Generate the linear decision problem

prob = LpProblem("Optimization", LpMaximize)

In [6]:
prob

Optimization:
MAXIMIZE
None
VARIABLES

In [9]:
# Create dicts of our variables

amount_invested = list(df_investment['Amount_Invested'])
returns = dict(zip(amount_invested,df_investment['Return']))
matuarity = dict(zip(amount_invested,df_investment['Matuarity']))
risks = dict(zip(amount_invested,df_investment['Risk']))
tax = dict(zip(amount_invested,df_investment['Tax-Free']))

In [10]:
# Define the decision variables, lowBound = 0 (decision variables >=0) and continuos variables

bond_vars = LpVariable.dicts("Amount_Invested", amount_invested, lowBound=0, cat='Continuous')
bond_vars

{'Xa': Amount_Invested_Xa,
 'Xb': Amount_Invested_Xb,
 'Xc': Amount_Invested_Xc,
 'Xd': Amount_Invested_Xd,
 'Xe': Amount_Invested_Xe}

In [11]:
# Set the decision function

prob += lpSum([returns[i]*bond_vars[i] for i in amount_invested])
prob

Optimization:
MAXIMIZE
0.095*Amount_Invested_Xa + 0.08*Amount_Invested_Xb + 0.09*Amount_Invested_Xc + 0.09*Amount_Invested_Xd + 0.09*Amount_Invested_Xe + 0.0
VARIABLES
Amount_Invested_Xa Continuous
Amount_Invested_Xb Continuous
Amount_Invested_Xc Continuous
Amount_Invested_Xd Continuous
Amount_Invested_Xe Continuous

In [12]:
# Constraint 1:
prob += lpSum([bond_vars[f] for f in amount_invested])==100000,'Total Available'

# Constrains 2:
prob += lpSum([risks[f]*bond_vars[f] for f in amount_invested]) <= 0.5*100000, "Risks"

# Constraint 3:
prob += lpSum([matuarity[f]*bond_vars[f] for f in amount_invested]) >= 0.5*100000, "Matuarity"

# Constraint 4:
prob += lpSum([tax[f]*bond_vars[f] for f in amount_invested]) >= 0.3*100000, "Tax"

# Constraint 5:
prob += lpSum([tax[f]*returns[f]*bond_vars[f] for f in amount_invested]) >= lpSum([returns[f]*bond_vars[f] for f in amount_invested])*0.4, "Return from Tax Free"


In [16]:
prob

Optimization:
MAXIMIZE
0.095*Amount_Invested_Xa + 0.08*Amount_Invested_Xb + 0.09*Amount_Invested_Xc + 0.09*Amount_Invested_Xd + 0.09*Amount_Invested_Xe + 0.0
SUBJECT TO
Total_Available: Amount_Invested_Xa + Amount_Invested_Xb + Amount_Invested_Xc
 + Amount_Invested_Xd + Amount_Invested_Xe = 100000

Risks: Amount_Invested_Xa + Amount_Invested_Xd + Amount_Invested_Xe <= 50000

Matuarity: Amount_Invested_Xb + Amount_Invested_Xe >= 50000

Tax: Amount_Invested_Xa + Amount_Invested_Xb + Amount_Invested_Xd >= 30000

Return_from_Tax_Free: 0.057 Amount_Invested_Xa + 0.048 Amount_Invested_Xb
 - 0.036 Amount_Invested_Xc + 0.054 Amount_Invested_Xd
 - 0.036 Amount_Invested_Xe >= 0

VARIABLES
Amount_Invested_Xa Continuous
Amount_Invested_Xb Continuous
Amount_Invested_Xc Continuous
Amount_Invested_Xd Continuous
Amount_Invested_Xe Continuous

In [17]:
prob.solve()

1

In [18]:
prob.writeLP("Optimization.lp")
print('The optimal result is:\n')
for v in prob.variables():
    if v.varValue>=0:
        print(v.name, '=', v.varValue)

The optimal result is:

Amount_Invested_Xa = 20338.983
Amount_Invested_Xb = 20338.983
Amount_Invested_Xc = 29661.017
Amount_Invested_Xd = 0.0
Amount_Invested_Xe = 29661.017
